# Concrete Crack Detection - Quick Start

This notebook demonstrates the crack detection pipeline.

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.models.unet import UNet
from src.models.deeplab import DeepLabV3Plus
from src.data.preprocessor import ImagePreprocessor
from src.utils.metrics import calculate_all_metrics

print("✓ Imports successful")

## 1. Initialize Models

In [ ]:
# Initialize U-Net
unet = UNet(n_channels=3, n_classes=1)
print(f"U-Net parameters: {sum(p.numel() for p in unet.parameters()):,}")

# Initialize DeepLab
deeplab = DeepLabV3Plus(n_channels=3, n_classes=1, pretrained=False)
print(f"DeepLab parameters: {sum(p.numel() for p in deeplab.parameters()):,}")

## 2. Test Preprocessing

In [ ]:
# Create sample image
sample_image = np.random.randint(0, 255, (512, 512, 3), dtype=np.uint8)

# Initialize preprocessor
preprocessor = ImagePreprocessor(target_size=(512, 512))

# Get transforms
train_transform = preprocessor.get_training_transforms()
val_transform = preprocessor.get_validation_transforms()

# Apply transforms
preprocessed = preprocessor.preprocess_single(sample_image, for_training=False)

print(f"Original shape: {sample_image.shape}")
print(f"Preprocessed shape: {preprocessed.shape}")
print(f"Preprocessed type: {type(preprocessed)}")

## 3. Test Inference

In [ ]:
# Create batch
batch = preprocessed.unsqueeze(0)
print(f"Batch shape: {batch.shape}")

# U-Net inference
unet.eval()
with torch.no_grad():
    unet_output = unet(batch)
    unet_pred = torch.sigmoid(unet_output) > 0.5

print(f"U-Net output shape: {unet_output.shape}")
print(f"U-Net prediction shape: {unet_pred.shape}")

# DeepLab inference
deeplab.eval()
with torch.no_grad():
    deeplab_output = deeplab(batch)
    deeplab_pred = torch.sigmoid(deeplab_output) > 0.5

print(f"DeepLab output shape: {deeplab_output.shape}")
print(f"DeepLab prediction shape: {deeplab_pred.shape}")

## 4. Metrics Calculation

In [ ]:
from src.utils.metrics import MetricsTracker

# Create dummy ground truth
dummy_gt = np.random.randint(0, 2, (512, 512), dtype=np.uint8)
dummy_pred = np.random.randint(0, 2, (512, 512), dtype=np.uint8)

# Calculate metrics
tracker = MetricsTracker()
tracker.update(dummy_pred, dummy_gt)

print(tracker.get_summary())

## 5. Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(sample_image)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(unet_pred.squeeze().cpu().numpy(), cmap='gray')
axes[1].set_title('U-Net Prediction')
axes[1].axis('off')

axes[2].imshow(deeplab_pred.squeeze().cpu().numpy(), cmap='gray')
axes[2].set_title('DeepLab Prediction')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Next Steps

1. Prepare your training data
2. Configure training parameters in `configs/`
3. Run training script: `python scripts/train.py`
4. Deploy API: `python -m uvicorn src.api.app:app`